In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Set to True to force a full refresh of the data
full_refresh = True

In [4]:
repo_root = os.popen('git rev-parse --show-toplevel').read().strip()

In [5]:
data_folder = os.path.join(repo_root, 'multi-parameter-estimation', 'data')

# Get list of data directories
data_dirs = os.listdir(data_folder)
data_dirs = [d for d in data_dirs if os.path.isdir(os.path.join(data_folder, d))]

# skip old-data
if 'old-data' in data_dirs:
    data_dirs.remove('old-data')

data_dirs.sort()
data_dirs

['2025-06-03--11h-02m-55s',
 '2025-06-03--11h-04m-33s',
 '2025-06-03--11h-05m-58s',
 '2025-06-03--11h-07m-21s',
 '2025-06-03--11h-08m-43s',
 '2025-06-03--11h-10m-06s',
 '2025-06-03--11h-11m-30s',
 '2025-06-03--11h-12m-54s',
 '2025-06-03--11h-14m-20s',
 '2025-06-03--11h-15m-44s',
 '2025-06-03--11h-17m-10s',
 '2025-06-03--11h-18m-35s',
 '2025-06-03--11h-20m-00s',
 '2025-06-03--11h-21m-24s',
 '2025-06-03--11h-22m-50s',
 '2025-06-03--11h-24m-16s',
 '2025-06-03--11h-25m-44s',
 '2025-06-03--11h-27m-10s',
 '2025-06-03--11h-28m-52s',
 '2025-06-03--11h-30m-17s',
 '2025-06-03--11h-31m-43s',
 '2025-06-03--11h-33m-09s',
 '2025-06-03--11h-34m-37s',
 '2025-06-03--11h-36m-03s',
 '2025-06-03--11h-37m-30s',
 '2025-06-03--11h-38m-56s',
 '2025-06-03--11h-40m-23s',
 '2025-06-03--11h-41m-53s',
 '2025-06-03--11h-43m-21s',
 '2025-06-03--11h-44m-49s',
 '2025-06-03--11h-46m-15s',
 '2025-06-03--11h-47m-43s',
 '2025-06-03--11h-49m-09s',
 '2025-06-03--11h-50m-33s',
 '2025-06-03--11h-51m-57s',
 '2025-06-03--11h-53

In [6]:
new_data_dirs = data_dirs.copy()

if not full_refresh:
    for d in data_dirs:
        if os.path.exists(os.path.join(data_folder, d, "scaled_coincidences.csv")):
            new_data_dirs.remove(d)

new_data_dirs

['2025-06-03--11h-02m-55s',
 '2025-06-03--11h-04m-33s',
 '2025-06-03--11h-05m-58s',
 '2025-06-03--11h-07m-21s',
 '2025-06-03--11h-08m-43s',
 '2025-06-03--11h-10m-06s',
 '2025-06-03--11h-11m-30s',
 '2025-06-03--11h-12m-54s',
 '2025-06-03--11h-14m-20s',
 '2025-06-03--11h-15m-44s',
 '2025-06-03--11h-17m-10s',
 '2025-06-03--11h-18m-35s',
 '2025-06-03--11h-20m-00s',
 '2025-06-03--11h-21m-24s',
 '2025-06-03--11h-22m-50s',
 '2025-06-03--11h-24m-16s',
 '2025-06-03--11h-25m-44s',
 '2025-06-03--11h-27m-10s',
 '2025-06-03--11h-28m-52s',
 '2025-06-03--11h-30m-17s',
 '2025-06-03--11h-31m-43s',
 '2025-06-03--11h-33m-09s',
 '2025-06-03--11h-34m-37s',
 '2025-06-03--11h-36m-03s',
 '2025-06-03--11h-37m-30s',
 '2025-06-03--11h-38m-56s',
 '2025-06-03--11h-40m-23s',
 '2025-06-03--11h-41m-53s',
 '2025-06-03--11h-43m-21s',
 '2025-06-03--11h-44m-49s',
 '2025-06-03--11h-46m-15s',
 '2025-06-03--11h-47m-43s',
 '2025-06-03--11h-49m-09s',
 '2025-06-03--11h-50m-33s',
 '2025-06-03--11h-51m-57s',
 '2025-06-03--11h-53

In [7]:
def load_coincidences(data_dir):
    coincidences = pd.read_csv(os.path.join(data_folder, data_dir, "labelled_coincidences.csv"))
    coincidences["data_dir"] = data_dir
    return coincidences

coincidences_df = pd.concat([load_coincidences(d) for d in new_data_dirs], ignore_index=True)
coincidences_df

,detector_a_name,detector_b_name,detector_a_time_tag,detector_b_time_tag,timestamp,tomography_setting_t,tomography_setting_r,data_dir,estimation_label
0,1,4,2448202930981002240,2448202930981002240,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H
1,1,4,2448202930981636608,2448202930981636608,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H
2,1,4,2448202930982159360,2448202930982159360,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H
3,9,12,2448202930982371840,2448202930982371840,2025-06-03--11h-03m-32s,H,H,2025-06-03--11h-02m-55s,DB_H
4,1,4,2448202930982511616,2448202930982511616,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H
...,...,...,...,...,...,...,...,...,...
27667306,7,10,2448249772831294464,2448249772831294464,2025-06-03--13h-05m-27s,V,H,2025-06-03--13h-04m-03s,C
27667307,7,11,2448249772832961536,2448249772832961536,2025-06-03--13h-05m-27s,V,H,2025-06-03--13h-04m-03s,C
27667308,9,12,2448249772833267200,2448249772833267200,2025-06-03--13h-05m-26s,V,H,2025-06-03--13h-04m-03s,DB_V
27667309,1,4,2448249772834622464,2448249772834622464,2025-06-03--13h-05m-26s,V,H,2025-06-03--13h-04m-03s,DB_H


In [8]:
# If there is no "coincidences" column, it is because we have line by line coincidences, so coincidences are 1
if "coincidences" not in coincidences_df.columns:
    coincidences_df["coincidences"] = 1

# If there is no "repetitions" column, it is because we have line by line coincidences, so repetitions are the index within the data_dir, tomography_settings
if "repetition" not in coincidences_df.columns:
    coincidences_df["repetition"] = coincidences_df.groupby(["data_dir", "tomography_setting_t", "tomography_setting_r"]).cumcount()
    
coincidences_df

,detector_a_name,detector_b_name,detector_a_time_tag,detector_b_time_tag,timestamp,tomography_setting_t,tomography_setting_r,data_dir,estimation_label,coincidences,repetition
0,1,4,2448202930981002240,2448202930981002240,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,0
1,1,4,2448202930981636608,2448202930981636608,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,1
2,1,4,2448202930982159360,2448202930982159360,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,2
3,9,12,2448202930982371840,2448202930982371840,2025-06-03--11h-03m-32s,H,H,2025-06-03--11h-02m-55s,DB_H,1,3
4,1,4,2448202930982511616,2448202930982511616,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,4
...,...,...,...,...,...,...,...,...,...,...,...
27667306,7,10,2448249772831294464,2448249772831294464,2025-06-03--13h-05m-27s,V,H,2025-06-03--13h-04m-03s,C,1,92009
27667307,7,11,2448249772832961536,2448249772832961536,2025-06-03--13h-05m-27s,V,H,2025-06-03--13h-04m-03s,C,1,92010
27667308,9,12,2448249772833267200,2448249772833267200,2025-06-03--13h-05m-26s,V,H,2025-06-03--13h-04m-03s,DB_V,1,92011
27667309,1,4,2448249772834622464,2448249772834622464,2025-06-03--13h-05m-26s,V,H,2025-06-03--13h-04m-03s,DB_H,1,92012


Double bunched events are only resolved half of the time. Therefore, we now throw away half of all non-double-bunched events to recover the expected statistics.

In [9]:
coincidences_df["scaled_coincidences"] = coincidences_df["coincidences"].astype(float)

coincidences_df.loc[coincidences_df["estimation_label"] == "SB", "scaled_coincidences"] = coincidences_df.loc[coincidences_df["estimation_label"] == "SB", "coincidences"] * 0.5
coincidences_df.loc[coincidences_df["estimation_label"] == "C", "scaled_coincidences"] = coincidences_df.loc[coincidences_df["estimation_label"] == "C", "coincidences"] * 0.5

coincidences_df

,detector_a_name,detector_b_name,detector_a_time_tag,detector_b_time_tag,timestamp,tomography_setting_t,tomography_setting_r,data_dir,estimation_label,coincidences,repetition,scaled_coincidences
0,1,4,2448202930981002240,2448202930981002240,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,0,1.0
1,1,4,2448202930981636608,2448202930981636608,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,1,1.0
2,1,4,2448202930982159360,2448202930982159360,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,2,1.0
3,9,12,2448202930982371840,2448202930982371840,2025-06-03--11h-03m-32s,H,H,2025-06-03--11h-02m-55s,DB_H,1,3,1.0
4,1,4,2448202930982511616,2448202930982511616,2025-06-03--11h-03m-33s,H,H,2025-06-03--11h-02m-55s,DB_H,1,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27667306,7,10,2448249772831294464,2448249772831294464,2025-06-03--13h-05m-27s,V,H,2025-06-03--13h-04m-03s,C,1,92009,0.5
27667307,7,11,2448249772832961536,2448249772832961536,2025-06-03--13h-05m-27s,V,H,2025-06-03--13h-04m-03s,C,1,92010,0.5
27667308,9,12,2448249772833267200,2448249772833267200,2025-06-03--13h-05m-26s,V,H,2025-06-03--13h-04m-03s,DB_V,1,92011,1.0
27667309,1,4,2448249772834622464,2448249772834622464,2025-06-03--13h-05m-26s,V,H,2025-06-03--13h-04m-03s,DB_H,1,92012,1.0


In [12]:
# Sum up by estimation label
df_sum = (
    coincidences_df.groupby(["data_dir", "estimation_label", "repetition"]) # Sum over the different tomography settings (HH, HV, VH, VV)
    .agg({"scaled_coincidences": "sum",})
    .sort_values(by=["data_dir", "repetition", "estimation_label"])
    .reset_index()
)
df_sum

,data_dir,estimation_label,repetition,scaled_coincidences
0,2025-06-03--11h-02m-55s,C,0,0.5
1,2025-06-03--11h-02m-55s,DB_H,0,3.0
2,2025-06-03--11h-02m-55s,DB_H,1,4.0
3,2025-06-03--11h-02m-55s,DB_H,2,4.0
4,2025-06-03--11h-02m-55s,C,3,0.5
...,...,...,...,...
13905757,2025-06-03--13h-04m-03s,DB_H,94004,1.0
13905758,2025-06-03--13h-04m-03s,C,94005,0.5
13905759,2025-06-03--13h-04m-03s,C,94006,0.5
13905760,2025-06-03--13h-04m-03s,C,94007,0.5


In [ ]:
df_pivoted_sum = df_sum.pivot(index=['data_dir', 'repetition'], columns='estimation_label', values='scaled_coincidences')
df_pivoted_sum = df_pivoted_sum.reset_index()  # optional, to flatten the index
df_pivoted_sum.columns.name = None  # remove the name of the columns
df_pivoted_sum = df_pivoted_sum.fillna(0)  # fill NaN values with 0
df_pivoted_sum

,data_dir,repetition,C,DB_H,DB_V,SB
0,2025-06-02--17h-06m-16s,0,0.0,1.0,0.0,0.0
1,2025-06-02--17h-06m-16s,1,0.0,1.0,0.0,0.0
2,2025-06-02--17h-06m-16s,2,0.0,1.0,0.0,0.0
3,2025-06-02--17h-06m-16s,3,0.0,1.0,0.0,0.0
4,2025-06-02--17h-06m-16s,4,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
78188,2025-06-02--17h-08m-19s,16151,0.0,0.0,0.0,0.5
78189,2025-06-02--17h-08m-19s,16152,0.0,1.0,0.0,0.0
78190,2025-06-02--17h-08m-19s,16153,0.0,0.0,1.0,0.0
78191,2025-06-02--17h-08m-19s,16154,0.0,0.0,0.0,0.5


In [ ]:
df_pivoted_sum["N"] = df_pivoted_sum["SB"] + df_pivoted_sum["C"] + df_pivoted_sum["DB_H"] + df_pivoted_sum["DB_V"]
df_pivoted_sum.head(10)

,data_dir,repetition,C,DB_H,DB_V,SB,N
0,2025-06-02--17h-06m-16s,0,0.0,1.0,0.0,0.0,1.0
1,2025-06-02--17h-06m-16s,1,0.0,1.0,0.0,0.0,1.0
2,2025-06-02--17h-06m-16s,2,0.0,1.0,0.0,0.0,1.0
3,2025-06-02--17h-06m-16s,3,0.0,1.0,0.0,0.0,1.0
4,2025-06-02--17h-06m-16s,4,0.0,1.0,0.0,0.0,1.0
5,2025-06-02--17h-06m-16s,5,0.0,1.0,0.0,0.0,1.0
6,2025-06-02--17h-06m-16s,6,0.0,1.0,0.0,0.0,1.0
7,2025-06-02--17h-06m-16s,7,0.0,1.0,0.0,0.0,1.0
8,2025-06-02--17h-06m-16s,8,0.0,1.0,0.0,0.0,1.0
9,2025-06-02--17h-06m-16s,9,0.0,1.0,0.0,0.0,1.0


In [ ]:
# save the dataframes to csv files based on the data_dir
for data_dir in new_data_dirs:
    df_subset = df_pivoted_sum[df_pivoted_sum["data_dir"] == data_dir]
    if not df_subset.empty:
        output_file = os.path.join(data_folder, data_dir, "scaled_coincidences.csv")
        df_subset.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-06m-16s/scaled_coincidences.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-07m-37s/scaled_coincidences.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-07m-47s/scaled_coincidences.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-07m-56s/scaled_coincidences.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-08m-05s/scaled_coincidences.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-par

In [ ]:
# sum all data
df_pivoted_sum_all = df_pivoted_sum.groupby("data_dir").sum().reset_index()
df_pivoted_sum_all

,data_dir,repetition,C,DB_H,DB_V,SB,N
0,2025-06-02--17h-06m-16s,18662995,0.0,6110.0,0.0,0.0,6110.0
1,2025-06-02--17h-07m-37s,92609245,250.0,13084.0,0.0,13.0,13347.0
2,2025-06-02--17h-07m-47s,95364955,260.0,13262.0,0.0,14.5,13536.5
3,2025-06-02--17h-07m-56s,92936161,258.0,13074.0,0.0,22.0,13354.0
4,2025-06-02--17h-08m-05s,110580756,245.5,9738.0,170.0,2236.5,12390.0
5,2025-06-02--17h-08m-19s,130500090,226.0,3654.0,2406.0,4822.0,11108.0


In [ ]:
# save the dataframes to csv files based on the data_dir
for data_dir in new_data_dirs:
    df_subset = df_pivoted_sum_all[df_pivoted_sum_all["data_dir"] == data_dir]
    if not df_subset.empty:
        output_file = os.path.join(data_folder, data_dir, "scaled_coincidences_all.csv")
        df_subset.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-06m-16s/scaled_coincidences_all.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-07m-37s/scaled_coincidences_all.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-07m-47s/scaled_coincidences_all.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-07m-56s/scaled_coincidences_all.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQL/projects/multi-parameter-estimation/multi-parameter-estimation/data/2025-06-02--17h-08m-05s/scaled_coincidences_all.csv
Saved /home/jh115/Heriot-Watt University Team Dropbox/RES_EPS_EMQ